In [38]:
import numpy as np
from tqdm import tqdm 

In [79]:
NUM_CELLS = 7
ESCAPE_CELL = 3
ESCAPE_PROBA = 0.5
GENERAL_ESCAPE_PROBA = 0.1
NUM_SAMPLES = 100000
SELF_TRANSITION = 0.3333

In [80]:
    def F(x2:int, x1: int) -> float:
        assert x1 >= 0 and x1 < NUM_CELLS
        assert x2 >= 0 and x2 < NUM_CELLS
        if x2 == x1:
            return SELF_TRANSITION
        if abs(x2 - x1) != 1:
            return 0.
        else:
            proba = ( 1- SELF_TRANSITION) / 2 
            if x1 == 0 or x1 == NUM_CELLS - 1:
                proba *= 2
            return proba 

In [81]:
def T(x: int) -> float:
    if x == ESCAPE_CELL:
        return ESCAPE_PROBA
    else:
        return GENERAL_ESCAPE_PROBA

In [82]:
memoize = { }
def Tstar(xstar: int, x0: int, k: int) -> float:
    if k == 0:
        return T(xstar) * float(xstar == x0)
    if (xstar, x0, k) not in memoize:
        sum = 0
        for i in range(NUM_CELLS):
            sum += T(xstar) * ((1-T(i))/ T(i)) * Tstar(i,x0,k-1)* F(xstar,i)
        memoize[(xstar, x0, k)] = sum 
    return memoize[(xstar, x0, k)]

In [83]:
hits = [np.zeros((NUM_CELLS, 1000)) for _ in range(NUM_CELLS)]

In [84]:
transition_matrix = np.zeros((NUM_CELLS, NUM_CELLS))
for i in range(NUM_CELLS):
    for j in range(NUM_CELLS):
        transition_matrix[j, i] = F(j, i)

In [85]:
def monte_carlo_sample(x: int) -> (int, int):
    k = 0
    while True:
        if T(x) > np.random.uniform():
            return x, k
        else:
            p = transition_matrix[:,x]
            x = int(np.random.choice(NUM_CELLS, p=p))
            k += 1

In [86]:
for i in range(NUM_CELLS):
    for _ in tqdm(range(NUM_SAMPLES)):
        xstar, k = monte_carlo_sample(i)
        hits[i][xstar, k] += 1

100%|██████████| 100000/100000 [00:18<00:00, 5299.67it/s]


In [87]:
for i in range(NUM_CELLS):
    hits[i] /= hits[i].sum()

In [88]:
hits[0][:,:10]

array([[9.870e-02, 2.991e-02, 2.747e-02, 1.829e-02, 1.490e-02, 1.226e-02,
        9.940e-03, 8.610e-03, 7.000e-03, 5.710e-03],
       [0.000e+00, 6.056e-02, 3.564e-02, 3.332e-02, 2.551e-02, 2.134e-02,
        1.744e-02, 1.523e-02, 1.308e-02, 1.054e-02],
       [0.000e+00, 0.000e+00, 1.801e-02, 1.623e-02, 1.500e-02, 1.331e-02,
        1.115e-02, 9.390e-03, 8.490e-03, 6.980e-03],
       [0.000e+00, 0.000e+00, 0.000e+00, 2.746e-02, 2.939e-02, 2.922e-02,
        2.632e-02, 2.327e-02, 2.115e-02, 1.747e-02],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 9.500e-04, 1.340e-03,
        1.400e-03, 1.260e-03, 1.250e-03, 1.250e-03],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 3.300e-04,
        4.500e-04, 7.300e-04, 6.900e-04, 1.070e-03],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        9.000e-05, 1.800e-04, 2.800e-04, 3.100e-04]])

In [89]:
predicted = [np.zeros((NUM_CELLS, 10000)) for _ in range(NUM_CELLS)]
for xstar in range(NUM_CELLS):
    for x0 in range(NUM_CELLS):
        for k in range(10000):
            predicted[x0][xstar, k] = Tstar(xstar, x0, k)

In [90]:
predicted[0][:,:10]

array([[1.00000000e-01, 2.99970000e-02, 2.70000001e-02, 1.88991900e-02,
        1.53896760e-02, 1.23926355e-02, 1.02138181e-02, 8.44777099e-03,
        7.02188602e-03, 5.84856546e-03],
       [0.00000000e+00, 6.00030000e-02, 3.59981998e-02, 3.24000001e-02,
        2.59193520e-02, 2.16535815e-02, 1.79455426e-02, 1.49586126e-02,
        1.24734107e-02, 1.04150081e-02],
       [0.00000000e+00, 0.00000000e+00, 1.80018000e-02, 1.61999999e-02,
        1.54801800e-02, 1.33798800e-02, 1.14890618e-02, 9.72404602e-03,
        8.19964593e-03, 6.89346210e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.70040502e-02,
        2.88014398e-02, 2.93715285e-02, 2.68108654e-02, 2.38462785e-02,
        2.07470823e-02, 1.78814429e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        9.00180014e-04, 1.23012299e-03, 1.42912420e-03, 1.45746159e-03,
        1.41583437e-03, 1.33129075e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
   

In [94]:
hits[0][:,:10] / predicted[0][:,:10]

array([[0.987     , 0.99709971, 1.0174074 , 0.96776635, 0.96818152,
        0.98929723, 0.9731914 , 1.01920376, 0.99688317, 0.97630779],
       [       nan, 1.00928287, 0.99004951, 1.02839506, 0.98420671,
        0.98551826, 0.97182907, 1.01814255, 1.04863059, 1.01200114],
       [       nan,        nan, 1.00045551, 1.00185186, 0.96898098,
        0.99477723, 0.9704883 , 0.96564742, 1.03541056, 1.01255362],
       [       nan,        nan,        nan, 1.0168845 , 1.0204351 ,
        0.99484097, 0.98169155, 0.97583361, 1.01942045, 0.97699051],
       [       nan,        nan,        nan,        nan, 1.05534447,
        1.08932197, 0.97962095, 0.86451678, 0.88287163, 0.93893839],
       [       nan,        nan,        nan,        nan,        nan,
        1.22191671, 0.99985002, 1.1920655 , 0.96293321, 1.37419725],
       [       nan,        nan,        nan,        nan,        nan,
               nan, 1.11077784, 1.12971755, 1.20940554, 1.08990832]])